### Contents

[Imports](#Imports)

[bottom](#bottom)

### Imports

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
from pprint import pprint
import sys 
import json 
import itertools 
from tqdm.auto import tqdm
from datetime import datetime, date, timedelta
from load_dotenv import load_dotenv
load_dotenv() # OPENAI_API_KEY

import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from IPython.display import display, clear_output

/Users/antonandreytsev/llm_zoomcamp_venv3.9.7/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
matplotlib.rcParams['figure.figsize'] = (7, 7)
sns.set_style('whitegrid')

---

In [4]:
import openai
from openai import OpenAI

In [5]:
# os.environ['OPENAI_API_KEY'] = 'xxxx'

### Q1. Prepare Documents
Import documents.json, read the file and prepare the dataset:

```python
import json

with open('documents.json', 'rt') as f_in:
    docs_raw = json.load(f_in)

documents = []

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

len(documents)
```

How many records we have in the pre-processed "documents"?

- 1000
- 1051
- 901
- 948

In [9]:
import requests 

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
docs_raw = docs_response.json()

documents = []

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

len(documents)

948

### bottom